In [ ]:
import pandas as pd
from functools import reduce
from pathlib import Path
import os

code_dir = Path(os.getcwd())

# directory where the data is stored
base_path = code_dir.parent / "data"

# File 2 CBCL data
file2_path = f"{base_path}/mh_p_cbcl.csv"

file2 = pd.read_csv(file2_path)

file2_baseline_filtered = file2[file2['eventname'] == 'baseline_year_1_arm_1']

#drop score
file2_drop = file2_baseline_filtered.loc[:, ~file2_baseline_filtered.columns.str.endswith(('_r', '_t', '_m', '_nm', '_nm_2', '___1'))]
#drop eventname
file2_drop = file2_drop.drop(columns=['eventname'])
file2_final_cleaned = file2_drop.dropna(axis=0)

C:\Users\Makka Papa\AppData\Local\Temp\ipykernel_3008\2214807639.py:14: DtypeWarning: Columns (124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200) have mixed types. Specify dtype option on import or set low_memory=False.
  file2 = pd.read_csv(file2_path)


In [7]:
file2_final_cleaned.to_csv(f'{base_path}/cbcl_data_remove_useless_items.csv')
